In [1]:
import csv 
from gpt_utility import read_json, read_csv 
from verifier_utility import *

In [1]:

create_experiment_jsonl('data/test_set_2.csv',read_json(),[25,50,100])

[]

In [ ]:
#Create COT fine_tuning batch
make_verifier_train_set()
training_df = read_csv('output/verifier_train.csv')
 
generate_cot_train(training_df)

In [ ]:
#Create NCOT fine_tuning batch
training_df = read_csv('output/verifier_train.csv')
 
generate_ncot_train(training_df)

**Fine-tuning for COT**

Change the file_path to output/ft_filter_cot.jsonl if training a COT verifier, change the file_path to output/ft_filter_ncot.jsonl if training a NCOT verifier.

In [3]:

import requests
import os
import json
#TODO: define the API key
 
with open('input/secret.json', 'r', encoding='UTF-8') as file:
    secret_json = json.load(file)
api_key = secret_json['OPENAI_API_KEY']
# client = OpenAI(api_key=api_key)

 
# Define headers for the upload request
headers = {
    "Authorization": f"Bearer {api_key}"
}

# File upload request URL
upload_url = "https://api.openai.com/v1/files"
file_path = 'output/ft_filter_cot.jsonl'  # Change this to the actual file path

# Open the file in binary mode
with open(file_path, 'rb') as f:
    # Make the file upload request
    response = requests.post(upload_url, headers=headers, files={
        'file': (os.path.basename(file_path), f, 'application/json')
    }, data={
        "purpose": "fine-tune"
    })

# Check if file upload was successful
if response.status_code == 200:
    upload_response = response.json()
    file_id = upload_response['id']  # The uploaded file ID
    print(f"File uploaded successfully! File ID: {file_id}")
else:
    print(f"File upload failed. Status code: {response.status_code}")
    print("Response:", response.text)

File uploaded successfully! File ID: file-BJkQig2CLVjB9ft5689dAk


In [4]:
import requests
import json
#TODO: update file_id to what is given to you by running the above script
#TODO: choose model between gpt-4o-mini-2024-07-18 or gpt-4o-2024-08-06 by changing the model field of fine_tune_data


headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}


fine_tune_url = "https://api.openai.com/v1/fine_tuning/jobs"

# Replace this with the file ID from the previous upload step
file_id = "file-BJkQig2CLVjB9ft5689dAk"


fine_tune_data = {
    "training_file": file_id, 


    "model": "gpt-4o-mini-2024-07-18",  
     "hyperparameters": {
         "n_epochs": 2, 
         "batch_size": 1,
            "learning_rate_multiplier": 1.8,

     }
}


response = requests.post(fine_tune_url, headers=headers, data=json.dumps(fine_tune_data))

# Check if the fine-tuning request was successful
if response.status_code == 200:
    print("Fine-tuning request submitted successfully!")
    print("Response:", response.json())
else:
    print(f"Failed to submit fine-tuning request. Status code: {response.status_code}")
    print("Response:", response.text)

Fine-tuning request submitted successfully!
Response: {'object': 'fine_tuning.job', 'id': 'ftjob-5kw1nkT1dzHtkiMlZmH8g7OH', 'model': 'gpt-4o-mini-2024-07-18', 'created_at': 1752794574, 'finished_at': None, 'fine_tuned_model': None, 'organization_id': 'org-okg2CRfiQn8iBsrlTRIZdan9', 'result_files': [], 'status': 'validating_files', 'validation_file': None, 'training_file': 'file-BJkQig2CLVjB9ft5689dAk', 'hyperparameters': {'n_epochs': 2, 'batch_size': 1, 'learning_rate_multiplier': 1.8}, 'trained_tokens': None, 'error': {}, 'user_provided_suffix': None, 'seed': 976191130, 'estimated_finish': None, 'integrations': [], 'metadata': None, 'usage_metrics': None, 'shared_with_openai': False, 'eval_id': None, 'method': {'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 1, 'learning_rate_multiplier': 1.8, 'n_epochs': 2}}}}


In [5]:
#TODO: change fine_tune_job_id to the actual id provided by above script
#TODO: add the api_key when running this script again after a few hours


headers = {
    "Authorization": f"Bearer {api_key}"
}

# Replace this with the actual fine-tuning job ID
fine_tune_job_id = "ftjob-5kw1nkT1dzHtkiMlZmH8g7OH" # Model Choice 2


url = f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tune_job_id}"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Fine-tuning job details retrieved successfully!")
    print("Response:", response.json())
else:
    print(f"Failed to retrieve fine-tuning job details. Status code: {response.status_code}")
    print("Response:", response.text)

Fine-tuning job details retrieved successfully!
Response: {'object': 'fine_tuning.job', 'id': 'ftjob-5kw1nkT1dzHtkiMlZmH8g7OH', 'model': 'gpt-4o-mini-2024-07-18', 'created_at': 1752794574, 'finished_at': None, 'fine_tuned_model': None, 'organization_id': 'org-okg2CRfiQn8iBsrlTRIZdan9', 'result_files': [], 'status': 'validating_files', 'validation_file': None, 'training_file': 'file-BJkQig2CLVjB9ft5689dAk', 'hyperparameters': {'n_epochs': 2, 'batch_size': 1, 'learning_rate_multiplier': 1.8}, 'trained_tokens': None, 'error': {}, 'user_provided_suffix': None, 'seed': 976191130, 'estimated_finish': None, 'integrations': [], 'metadata': None, 'usage_metrics': None, 'shared_with_openai': False, 'eval_id': None, 'method': {'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 2, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}}}
